# Di-lepton analysis

### Introduction

This notebook is used to process the data stored in [NANOAOD](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD) format, and select candidate events of exclusive dilepton production, $pp\to p\oplus\ell\ell\oplus p$, with $\ell\in\{ e,\mu,\tau \} $. Feynman diagram of this process is shown bellow: 

<img src="img/diagrams.png" alt="Feynman diagrams" width="800">



This notebook was prepared based on the [df102_NanoAODDimuonAnalysis.py](https://root.cern.ch/doc/master/group__tutorial__dataframe.html) example from ROOT.

In [1]:
import ROOT
 
# Enable multi-threading
ROOT.ROOT.EnableImplicitMT()

#include auxiliary functions from C++ header
ROOT.gInterpreter.Declare('#include "SelectorTools.h"')

Welcome to JupyROOT 6.24/00


True

### Datasets
`Datasets.py` contains the list of datasets that can be used in the analysis. The `listDatasets()` command will print out the complete list of datasets and also information about samples availability on EOS. 

In [2]:
from Datasets import *
listDatasets()

The following datasets are available:
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-Pilot_106X_upgrade2018_realistic_v15_L1v1-v2/NANOAODSIM (No directories available in EOS for this dataset)
/DYToMuMu_pomflux_Pt-30_TuneCP5_13TeV-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM (No directories available in EOS for this dataset)
/SingleMuon/Run2017A-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD (No directories available in EOS for this dataset)
/SingleMuon/Run2017B-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD (in EOS: nfiles = 33, size = 56.40GB)
/SingleMuon/Run2017C-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD (No directories available in EOS for this dataset)
/SingleMuon/Run2017D-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD (No directories available in EOS for this dataset)
/SingleMuon/Run2017E-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD (No directories available in EOS for this dataset)
/SingleMuon/Run2017F-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD (No di

### Analysis

Measurement of exclusive production of lepton pairs rely on two selections:
1. Exclusive cuts - leptons are produced exclusively, i.e., no other particles produced during the proton-proton interaction
2. Correlation between leptons and protons: Due to energy-momentum conservation, the following equation holds:
$ \xi_\pm = \frac{1}{\sqrt{s}}\left[ p_{T,\ell1}\cdot e^{\pm\eta_{\ell1}} + p_{T,\ell2}\cdot e^{\pm\eta_{\ell2}} \right] $, where $\xi$ is proton momentum loss, and $\pm$ sign related to $\xi$ reconstruction of positive/negative proton.

The analysis strategy is to apply selection cuts as in (1) and then plot the difference between $\xi$ reconstructed from leptons and $\xi$ measured by PPS. The events where di-lepton kinematics agree with proton kinematics will be candidates for exclusive dilepton production.

---
Proton content is not available in MC yet, so here we will analyze the data, searching for the correlation between measured $\xi$ from the proton and the predicted $\xi$ based on the lepton kinematics. We are using data available on EOS to speed up the analysis. 

For the example, we will process a small amount of data:

In [3]:
#Select which datasets to use
datasets=[
    'Run2017B_SingleMuon',
#    'Run2018A_EGamma',
#    'Run2018B_DoubleMuon',
#    'Run2018C_DoubleMuon',
#    'Run2018D_DoubleMuon',
]

In [5]:
#Create a list of files to analyze
for ds in datasets:
    listFiles=getFilesForDataset(ds)
files = ROOT.std.vector("string")(len(listFiles))
for i,f in enumerate(listFiles):
    files[i]=f  
print('found %d files'%len(files))

found 33 files


To read the data we will use [RDataFrame](https://root.cern.ch/doc/master/group__tutorial__dataframe.html) interface to work with nanoAOD trees.

In [6]:
df = ROOT.RDataFrame("Events",files)

In [7]:
#print number of events in the dataframe:
nentries=df.Count()
print("%s entries in the dataframe" %nentries.GetValue())

75795703 entries in the dataframe


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class __pair_base<edm::Hash<1>,edm::ParameterSetBlob> is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


# 1. Exclusive cuts

## Event selection

We will skim the dataframe following event selection of the search for exclusive production of muon pairs ([JHEP07(2018)153](https://arxiv.org/abs/1803.04496))

The following cells contain two selection codes:
1. Preselection: which are contain selection cuts that are usually not used for the optimization
2. Selection cuts: These cuts are optimized and are dependent on the preselection cuts.

In [8]:
def preselect_dimuon(df):
    
    #trigger requirement
    trigger_bits='HLT_IsoMu27'
#    trigger_bits='HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8 || HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass8'
    df = df.Filter(trigger_bits,'Trigger')
    
    #muon selection
    good_mu  = 'Muon_pt>50 && abs(Muon_eta)<2.1' #kinematics
    good_mu += '&& Muon_tightId>0 && abs(Muon_pfRelIso04_all)<0.15' #identification + isolation
#    good_mu += '&& Muon_sip3d<4 && abs(Muon_dxy)<0.5 && abs(Muon_dz)<1.0' #vertex association
    df = df.Define('good_mu',good_mu).Filter('Sum(good_mu)==2','Two good muons')
    
    #save the good-muon related varialbes
    for attr in ['pt','eta','phi','charge','mass']:
        df=df.Define('good_mu_{}'.format(attr),'Muon_{}[good_mu]'.format(attr))
#    df = df.Define("Acoplanarity", "abs(acos(cos(good_mu_phi[0]-good_mu_phi[1])))")
#    df = df.Define("mll", "InvariantMass(good_mu_pt, good_mu_eta, good_mu_phi, good_mu_mass)")
   
    return df

def select_dimuon(df):
    
    #charge muon1 != charge muon 2
    df = df.Filter('good_mu_charge[0]!=good_mu_charge[1]','OS Muons')
    
    # Acoplanarity cut (back-to-back muons)
    df = df.Filter('(1-abs(acos(cos(good_mu_phi[0]-good_mu_phi[1])))/M_PI)<0.09','Acoplanarity')
    
    # Mass cut
    df = df.Filter('InvariantMass(good_mu_pt, good_mu_eta, good_mu_phi, good_mu_mass)>110','mll cut')

    # Track selection
    ### Missing for now  ###
    
    return df

In [9]:
#preselection cuts:
df_dimu_presel = preselect_dimuon(df)

In [10]:
# Request cut-flow report
report = df_dimu_presel.Report()
report.Print()

Trigger   : pass=31909272   all=75795703   -- eff=42.10 % cumulative eff=42.10 %
Two good muons: pass=56253      all=31909272   -- eff=0.18 % cumulative eff=0.07 %


## Pre-selection plots

Before applying analysis cuts here are a few distributions we obtaind from the data:

### acoplanarity

Defined as $A = 1 - |\phi_{\ell,1}-\phi_{\ell,2}|/\pi=1-|\Delta\phi_{\ell\ell}|/\pi$. In exclusive production, both leptons should be back top back in the absence of other particles due to momentum conservation, then $A\sim0$

In [11]:
df_dimu_presel_acopl = df_dimu_presel.Define("Acoplanarity", "1-abs(acos(cos(good_mu_phi[0]-good_mu_phi[1])))/M_PI")

In [12]:
# make histograms
h_acopl = df_dimu_presel_acopl.Histo1D(("Acoplanarity", ";Acoplanarity [rad];N_{Events}", 50, 0, 1), "Acoplanarity")

In [13]:
# Produce plots
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)

h_acopl.GetXaxis().SetTitleSize(0.04)
h_acopl.GetYaxis().SetTitleSize(0.04)
h_acopl.Draw()

label = ROOT.TLatex(); label.SetNDC(True)
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Work in progress}")
label.SetTextSize(0.030); label.DrawLatex(0.790, 0.920, "#sqrt{s} = 13 TeV")

c.SaveAs("dimuon_acoplanarity.pdf")

Info in <TCanvas::Print>: pdf file dimuon_acoplanarity.pdf has been created


### invariant mass

The measurement is performed above the Z mass peak. Invariant mass computed from the two good muons can be obtained using the [ROOT::VecOps::InvariantMass()](https://root.cern/doc/master/group__vecops.html#ga2c531eae910edad48bbf7319cc6d7e58) function

In [14]:
df_dimu_presel_mass = df_dimu_presel.Define("mll", "InvariantMass(good_mu_pt, good_mu_eta, good_mu_phi, good_mu_mass)")
h_mll = df_dimu_presel_mass.Histo1D(("Dimuon_mass", ";m_{#mu#mu} [GeV];N_{Events}", 75, 50, 300), "mll")

In [15]:
# Produce plots
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)

h_mll.GetXaxis().SetTitleSize(0.04)
h_mll.GetYaxis().SetTitleSize(0.04)
h_mll.Draw()

label = ROOT.TLatex(); label.SetNDC(True)
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Work in progress}")
label.SetTextSize(0.030); label.DrawLatex(0.790, 0.920, "#sqrt{s} = 13 TeV")
 
c.SaveAs("dimuon_spectrum.pdf")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file dimuon_spectrum.pdf has been created


# Final event selection

Based on analysis of signal and background distributions obtained in the previous step, final analysis cuts are selected and applied to the dataset

In [16]:
#event selection:
df_dimu_sel = select_dimuon(df_dimu_presel)

In [17]:
# Request cut-flow report
report = df_dimu_sel.Report()
report.Print()

Trigger   : pass=31909272   all=75795703   -- eff=42.10 % cumulative eff=42.10 %
Two good muons: pass=56253      all=31909272   -- eff=0.18 % cumulative eff=0.07 %
OS Muons  : pass=56147      all=56253      -- eff=99.81 % cumulative eff=0.07 %
Acoplanarity: pass=19449      all=56147      -- eff=34.64 % cumulative eff=0.03 %
mll cut   : pass=16854      all=19449      -- eff=86.66 % cumulative eff=0.02 %


# 2. Correlation between leptons and protons

## Tagged protons selection

Once events with exclusive signature have been selected, correlation between the proton kimenatics and lepton kinematics is tested. In exclusive production where leptons and protons have the same origin, full correlation will be observed.

Before starting with the analysis, we will select events with good tagged protons and central event kinematics (di-$\ell$) which predict a proton within the acceptance of the PPS.

### PPS acceptance:

We will extract the acceptance of PPS detectors

In [26]:
def select_proton_pos(df):
    
    #proton selection (will work only with 1 tagger proton per arm - 2017 data)
    df = df.Define('xi_pr0','Sum((Proton_multiRP_arm==0)*Proton_multiRP_xi)').Filter('xi_pr0>0','proton in the positive arm') # proton in positive direction (+z)
    
    return df

def select_proton_neg(df):
    
    #proton selection (will work only with 1 tagger proton per arm - 2017 data)
    df = df.Define('xi_pr1','Sum((Proton_multiRP_arm==1)*Proton_multiRP_xi)').Filter('xi_pr1>0','proton in the negative arm') # proton in negative direction (-z)
    
    return df

In [27]:
df_dimu_sel_pro0 = select_proton_pos(df_dimu_sel)

minVal0 = df_dimu_sel_pro0.Min('xi_pr0')
maxVal0 = df_dimu_sel_pro0.Max('xi_pr0')
print("Positive arm acceptance:%s <= xi <= %s" %(minVal0.GetValue(), maxVal0.GetValue()))
 

Positive arm acceptance:0.024318695068359375 <= xi <= 0.170318603515625


In [32]:
df_dimu_sel_pro0_withll = df_dimu_sel_pro0.Define("xi0_ll", "Sum(good_mu_pt*exp(+good_mu_eta))/13000.")
df_dimu_sel_pro0_withll = df_dimu_sel_pro0_withll.Filter('xi0_ll>%s && xi0_ll<%s'%(minVal0.GetValue(), maxVal0.GetValue()),'dilep predict a proton in +z')

In [33]:
report = df_dimu_sel_pro0_withll.Report()
report.Print()

Trigger   : pass=31909272   all=75795703   -- eff=42.10 % cumulative eff=42.10 %
Two good muons: pass=56253      all=31909272   -- eff=0.18 % cumulative eff=0.07 %
OS Muons  : pass=56147      all=56253      -- eff=99.81 % cumulative eff=0.07 %
Acoplanarity: pass=19449      all=56147      -- eff=34.64 % cumulative eff=0.03 %
mll cut   : pass=16854      all=19449      -- eff=86.66 % cumulative eff=0.02 %
proton in positive direction: pass=1591       all=16854      -- eff=9.44 % cumulative eff=0.00 %
dilep predict a proton in +z: pass=486        all=1591       -- eff=30.55 % cumulative eff=0.00 %


In [34]:
df_dimu_sel_pro1 = select_proton_neg(df_dimu_sel)

minVal1 = df_dimu_sel_pro1.Min('xi_pr1')
maxVal1 = df_dimu_sel_pro1.Max('xi_pr1')
print("Negative arm acceptance:%s <= xi <= %s" %(minVal1.GetValue(), maxVal1.GetValue()))
 

Negative arm acceptance:0.026493072509765625 <= xi <= 0.17578125


In [35]:
df_dimu_sel_pro1_withll = df_dimu_sel_pro1.Define("xi1_ll", "Sum(good_mu_pt*exp(-good_mu_eta))/13000.")
df_dimu_sel_pro1_withll = df_dimu_sel_pro1_withll.Filter('xi1_ll>%s  && xi1_ll<%s'%(minVal1.GetValue(), maxVal1.GetValue()),'dilep predict a proton in -z')

In [36]:
report = df_dimu_sel_pro1_withll.Report()
report.Print()

Trigger   : pass=31909272   all=75795703   -- eff=42.10 % cumulative eff=42.10 %
Two good muons: pass=56253      all=31909272   -- eff=0.18 % cumulative eff=0.07 %
OS Muons  : pass=56147      all=56253      -- eff=99.81 % cumulative eff=0.07 %
Acoplanarity: pass=19449      all=56147      -- eff=34.64 % cumulative eff=0.03 %
mll cut   : pass=16854      all=19449      -- eff=86.66 % cumulative eff=0.02 %
proton in negative direction: pass=1724       all=16854      -- eff=10.23 % cumulative eff=0.00 %
dilep predict a proton in -z: pass=477        all=1724       -- eff=27.67 % cumulative eff=0.00 %


Once the PPS acceptance is extracted, and we selected events where two-lepton kinematics suggest a proton within the PPS detector, we can inspect correlations.

In [49]:
df_dimu_sel_pro0_withll_delXi = df_dimu_sel_pro0_withll.Define('DelXi','xi0_ll-xi_pr0')
h_xi0 = df_dimu_sel_pro0_withll_delXi.Histo1D(("Delta_xi0", ";#xi_{ll}-#xi_{p+}};N_{Events}", 100, -0.1, 0.1), "DelXi")

In [50]:
# Produce plots
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)

h_xi0.GetXaxis().SetTitleSize(0.04)
h_xi0.GetYaxis().SetTitleSize(0.04)
h_xi0.Draw()

label = ROOT.TLatex(); label.SetNDC(True)
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Work in progress}")
label.SetTextSize(0.030); label.DrawLatex(0.790, 0.920, "#sqrt{s} = 13 TeV")
 
c.SaveAs("correlation_positive_arm.pdf")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file correlation_positive_arm.pdf has been created


In [46]:
df_dimu_sel_pro1_withll_delXi = df_dimu_sel_pro1_withll.Define('DelXi','xi1_ll-xi_pr1')
h_xi1 = df_dimu_sel_pro1_withll_delXi.Histo1D(("Delta_xi1", ";#xi_{ll}-#xi_{p-}};N_{Events}", 100, -0.1, 0.1), "DelXi")

In [47]:
 # Produce plots
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)

h_xi1.GetXaxis().SetTitleSize(0.04)
h_xi1.GetYaxis().SetTitleSize(0.04)
h_xi1.Draw()

label = ROOT.TLatex(); label.SetNDC(True)
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Work in progress}")
label.SetTextSize(0.030); label.DrawLatex(0.790, 0.920, "#sqrt{s} = 13 TeV")
 
c.SaveAs("correlation_negative_arm.pdf")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file correlation_negative_arm.pdf has been created


In [55]:
h_xi1_2D = df_dimu_sel_pro1_withll.Histo2D(("negXi_2D", ";#xi_{ll};#xi_{pp}", 64, 0., 0.2, 32, 0., 0.2), "xi1_ll", "xi_pr1");

In [ ]:
 # Produce plots
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)

h_xi1_2D.GetXaxis().SetTitleSize(0.04)
h_xi1_2D.GetYaxis().SetTitleSize(0.04)
h_xi1_2D.Draw('colz')

label = ROOT.TLatex(); label.SetNDC(True)
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Work in progress}")
label.SetTextSize(0.030); label.DrawLatex(0.790, 0.920, "#sqrt{s} = 13 TeV")
 
c.SaveAs("correlation_negative_arm_2D.pdf")